In [1]:
import pandas as pd
import seaborn as sns
import datetime
import numpy as np
from matplotlib import pyplot as plt

#%matplotlib tk

In [2]:
legco_df = pd.read_csv('records-all-with-info.csv')

In [3]:
legco_df.describe()

,Unnamed: 0
count,161700.000000
mean,80849.500000
std,46678.913601
min,0.000000
25%,40424.750000
50%,80849.500000
75%,121274.250000
max,161699.000000


In [4]:
legco_df.head()

,Unnamed: 0,topic_id,member_id,vote,vote-date,vote-time,motion-en,mover-en,mover-type,vote-separate-mechanism
0,0,17/10/2012-1,曾鈺成,Present,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes
1,1,17/10/2012-1,何俊仁,Yes,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes
2,2,17/10/2012-1,李卓人,Yes,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes
3,3,17/10/2012-1,涂謹申,Yes,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes
4,4,17/10/2012-1,陳鑑林,No,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes


In [5]:
def from_str_to_date(date_str):
    t = date_str.split('/')
    day = int(t[0])
    month = int(t[1])
    year = int(t[2])
    dt = datetime.datetime(year, month, day)
    return dt

legco_df['dt'] = legco_df['vote-date'].apply(from_str_to_date)
legco_df.head()

,Unnamed: 0,topic_id,member_id,vote,vote-date,vote-time,motion-en,mover-en,mover-type,vote-separate-mechanism,dt
0,0,17/10/2012-1,曾鈺成,Present,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17
1,1,17/10/2012-1,何俊仁,Yes,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17
2,2,17/10/2012-1,李卓人,Yes,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17
3,3,17/10/2012-1,涂謹申,Yes,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17
4,4,17/10/2012-1,陳鑑林,No,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17


In [6]:
len(legco_df.member_id.unique())

71

In [7]:
def clean_record(t):
    if t == 'Dr Joseph LEE':
        t = 'Prof Joseph LEE'
    # https://github.com/code4hk/legcohk/issues/1
    if t == '郭偉强':
        t = '郭偉強'
    # Other normalization if any
    # ...
    return t
legco_df.member_id = legco_df.member_id.apply(clean_record)
len(legco_df.member_id.unique())

70

In [8]:
def to_numeric(x):
    if x == 'Yes':
        return 1
    elif x == 'No':
        return -1
    else:
        return 0
legco_df['vote'] = legco_df['vote'].apply(to_numeric)

In [9]:
legco_df.head()

,Unnamed: 0,topic_id,member_id,vote,vote-date,vote-time,motion-en,mover-en,mover-type,vote-separate-mechanism,dt
0,0,17/10/2012-1,曾鈺成,0,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17
1,1,17/10/2012-1,何俊仁,1,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17
2,2,17/10/2012-1,李卓人,1,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17
3,3,17/10/2012-1,涂謹申,1,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17
4,4,17/10/2012-1,陳鑑林,-1,17/10/2012,19:37:53,AMENDMENT BY DR HON KENNETH CHAN TO HON IP KIN...,Dr Kenneth CHAN,Member,Yes,2012-10-17


In [10]:
legco_df_selected = legco_df[['topic_id', 'member_id', 'vote', 'dt']]
legco_df_selected.head()

,topic_id,member_id,vote,dt
0,17/10/2012-1,曾鈺成,0,2012-10-17
1,17/10/2012-1,何俊仁,1,2012-10-17
2,17/10/2012-1,李卓人,1,2012-10-17
3,17/10/2012-1,涂謹申,1,2012-10-17
4,17/10/2012-1,陳鑑林,-1,2012-10-17


In [16]:
def get_pc1_from_df(df):
    df_matrix = pd.DataFrame(index=df['member_id'].unique())
    for gn, g in df.groupby('topic_id'):
        df_matrix[gn] = g.set_index('member_id')['vote']
    
    #NOTE: 
    #    For more rigorous study, 
    #    you need to investigate what are those missing values
    df_matrix.fillna(0)
    def clean_values(x):
        x[(x != 1) & (x != -1)] = 0
    df_matrix.apply(clean_values)
    
    X = np.matrix(df_matrix.as_matrix()).astype('float')
    X = X - np.mean(X, 0)
 
    from sklearn.decomposition import PCA
    pca = PCA(n_components=1)
    # X is of dimensions: n_samples x n_features
    X_reduced = pca.fit_transform(X)
    df_pc1 = pd.DataFrame(X_reduced, index=df_matrix.index, columns=['PC1'])
    df_pc1 = df_pc1.sort('PC1')
    #plt.figure(figsize=(12, 20))
    #plt.plot(df_pc1['PC1'], df_pc1.rank()['PC1'], 'd', markersize=10)
    #for (_x, _y, _s) in zip(df_pc1['PC1'], df_pc1.rank()['PC1'], df_pc1.index):
    #    plt.annotate(_s.decode('utf-8'), (_x, _y), xytext=(_x + 0.01, _y - 0.02))
    #plt.title('Spectrum from Principal Component 1')
    #plt.show()
    if df_pc1.T['梁國雄'][0] > df_pc1.T['曾鈺成'][0]:
        # Fix the relative order of landmark nodes
        df_pc1['PC1'] = - df_pc1['PC1'] 
    return df_pc1

df_pc1 = get_pc1_from_df(legco_df_selected)

In [12]:
print legco_df_selected.dt.max()
print legco_df_selected.dt.min()

2015-02-13 00:00:00
2012-10-17 00:00:00


In [13]:
import time
#from multiprocessing import Pool 
# I have 4 cores. Change this to fit you
#p = Pool(4)

from dateutil.relativedelta import relativedelta 

init_date = datetime.datetime(2012, 12, 26)
cutoff_dates = []
cur_date = init_date
while cur_date < legco_df_selected.dt.max():
    legco_df_selected_cut = legco_df_selected[legco_df_selected.dt < cur_date]
    cur_date += relativedelta(months=1)
    cutoff_dates.append(cur_date)
cutoff_dates.append(cur_date)

def get_1year_records_till(t_end):
    t_begin = t_end - relativedelta(years=1)
    return legco_df_selected[(legco_df_selected.dt >= t_begin) & (legco_df_selected.dt < t_end)]

def get_pc1_at_cutoff_date(t):
    return (t, get_pc1_from_df(get_1year_records_till(t)))

_begin_time = time.time()
# This runs for: 47.6116089821 
PCs = map(lambda t: (t, get_pc1_from_df(get_1year_records_till(t))), cutoff_dates)
# This runs for
#PCs = p.map(get_pc1_at_cutoff_date, cutoff_dates)
_end_time = time.time()
print 'Elapse:', _end_time - _begin_time

scores = pd.DataFrame(index=legco_df_selected.member_id.unique())
for (t, PC) in PCs:
    scores[t] = PC

rankings = scores.rank()
rankings.T['梁國雄']

Elapse: 44.2592389584


/usr/local/lib/python2.7/site-packages/matplotlib/pyplot.py:424: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


2013-01-26    21
2013-02-26    24
2013-03-26    24
2013-04-26    24
2013-05-26     1
2013-06-26     1
2013-07-26     1
2013-08-26     1
2013-09-26     1
2013-10-26     1
2013-11-26     1
2013-12-26     1
2014-01-26     1
2014-02-26     1
2014-03-26     1
2014-04-26     1
2014-05-26     4
2014-06-26     1
2014-07-26     1
2014-08-26     1
2014-09-26     1
2014-10-26     1
2014-11-26     1
2014-12-26     1
2015-01-26     1
2015-02-26     1
Name: 梁國雄, dtype: float64

In [21]:
rankings.T.describe()

,曾鈺成,何俊仁,李卓人,涂謹申,陳鑑林,梁耀忠,劉皇發,劉慧卿,譚耀宗,石禮謙,...,葉建源,葛珮帆,廖長江,潘兆平,鄧家彪,蔣麗芸,盧偉國,鍾國斌,鍾樹根,謝偉銓
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,...,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,29.153846,16.538462,11.538462,18.769231,58.576923,16.423077,44.076923,13.500000,65.461538,38.769231,...,16.538462,54.307692,48.692308,54.192308,49.038462,46.076923,67.538462,46.384615,43.730769,58.076923
std,3.002563,3.127053,2.420426,3.723522,7.642895,7.782920,6.632786,5.368426,6.034771,10.726818,...,2.564851,7.604048,4.937767,11.668828,8.567290,3.772777,1.333590,8.542023,10.286137,2.261381
min,26.000000,12.000000,8.000000,12.000000,42.000000,6.000000,36.000000,4.000000,47.000000,31.000000,...,13.000000,38.000000,46.000000,31.000000,29.000000,40.000000,65.000000,40.000000,35.000000,55.000000
25%,28.000000,14.000000,10.000000,16.250000,58.000000,6.250000,39.250000,10.250000,66.000000,31.000000,...,15.000000,52.250000,46.000000,52.000000,51.000000,42.000000,67.000000,40.250000,35.000000,56.000000
50%,28.000000,16.500000,12.000000,20.000000,59.500000,19.500000,40.000000,14.000000,68.000000,35.500000,...,16.000000,54.500000,47.000000,54.000000,52.000000,47.500000,68.000000,44.000000,37.000000,59.000000
75%,29.000000,19.000000,13.000000,22.750000,63.000000,22.750000,53.000000,16.750000,69.000000,38.000000,...,19.000000,62.000000,47.000000,66.000000,53.750000,48.000000,68.000000,46.000000,55.000000,60.000000
max,36.000000,22.000000,16.000000,23.000000,67.000000,25.000000,53.000000,22.000000,70.000000,63.000000,...,21.000000,62.000000,60.000000,68.000000,56.000000,53.000000,70.000000,66.000000,59.000000,62.000000


In [14]:
rankings.T['梁國雄']

2013-01-26    21
2013-02-26    24
2013-03-26    24
2013-04-26    24
2013-05-26     1
2013-06-26     1
2013-07-26     1
2013-08-26     1
2013-09-26     1
2013-10-26     1
2013-11-26     1
2013-12-26     1
2014-01-26     1
2014-02-26     1
2014-03-26     1
2014-04-26     1
2014-05-26     4
2014-06-26     1
2014-07-26     1
2014-08-26     1
2014-09-26     1
2014-10-26     1
2014-11-26     1
2014-12-26     1
2015-01-26     1
2015-02-26     1
Name: 梁國雄, dtype: float64

In [25]:
rankings.T.plot()
plt.show()

In [41]:
rankings_var = pd.DataFrame(rankings.T.var(), columns=['var'])

In [ ]:
var_kings = list(rankings_var.sort('var')[-10:-1].index)
print var_kings
rankings.T[var_kings].plot()
plt.show()

In [ ]:
len(legco_df_selected['member_id'].unique())

In [ ]:
d1 = df_pc1.rank()
d2 = df_pc1
d2['PC1'] = -d2['PC1']
d2 = d2.rank()
d1['another'] = d2['PC1']
d1
df_pc1.T['梁國雄']
df_pc1.T['曾鈺成'][0]

In [ ]:
df_matrix = pd.DataFrame(index=legco_df_selected['member_id'].unique())
for gn, g in legco_df_selected.groupby('topic_id'):
    df_matrix[gn] = g.set_index('member_id')['vote']
df_matrix.head(2)

In [ ]:
print df_matrix.shape
print np.product(df_matrix.shape)
print len(legco_df_selected)

In [ ]:
df_matrix.fillna(0)
df_matrix.describe().T.describe()

In [ ]:
df_matrix.describe()